In [1]:
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_VISIBLE_DEVICES=3


In [2]:
# -*- coding: utf-8 -*-
import dataset
# import keys_union
# import keys_keras
import numpy as np
import torch
import time
import os, sys
sys.path.insert(0, os.getcwd())
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from keras.callbacks import TensorBoard
from keras.utils import plot_model


Using TensorFlow backend.


In [17]:
characters = ' 0123456789-,./%￥|'
# characters = keys_keras.alphabet[:]
from model import get_model
nclass = len(characters) + 1
trainroot = '../data/lmdb/train_num_no_char_v2/'
valroot = '../data/lmdb/valid_num_no_char_v2/'
# modelPath = '../pretrain-models/keras.hdf5'
modelPath = ''
modelPath = '/mnt/wuwenhui/git_ocr_project/keras_crnn/save_model/num_char_000077_10.h5'
workers = 4
imgH = 32
imgW = 256
keep_ratio = False
random_sample = False
batchSize = 32

testSize = 128
n_len = 10
loss = 100000
interval = 200
LEARNING_RATE = 0.001
Learning_decay_step = 5000
PERCEPTION = 0.25
EPOCH_NUMS = 100000
MODEL_PATH = '/mnt/wuwenhui/git_ocr_project/keras_crnn/save_model'

LOG_FILE = 'log.txt'
SUMMARY_PATH = './log/'
if not os.path.exists(MODEL_PATH):
    print('Creating save model path!!')
    os.makedirs(MODEL_PATH)
if not os.path.exists(SUMMARY_PATH):
    os.makedirs(SUMMARY_PATH)

model, basemodel = get_model(
    height=imgH, nclass=nclass, learning_rate=LEARNING_RATE)

config = tf.ConfigProto(intra_op_parallelism_threads=4)
config.gpu_options.per_process_gpu_memory_fraction = PERCEPTION
KTF.set_session(tf.Session(config=config))

# 加载预训练参数
if os.path.exists(modelPath):
    # basemodel.load_weights(modelPath)
    model.load_weights(modelPath,by_name=True)
    print('model loaded')

# plot_model(basemodel, to_file='basemodel.png')
# plot_model(model, to_file='model.png')


def one_hot(text, length=10, characters=characters):
    label = np.zeros(length)
    for i, char in enumerate(text):
        index = characters.find(char)
        if index == -1:
            index = characters.find(u' ')
        if i < length:
            label[i] = index
    return label


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 32, None, 1)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, None, 64) 640         the_input[0][0]                  
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 16, None, 64) 0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 16, None, 128 73856       pool1[0][0]                      
__________________________________________________________________________________________________
pool2 (Max

In [18]:

# 导入数据
if random_sample:
    sampler = dataset.randomSequentialSampler(train_dataset, batchSize)
else:
    sampler = None
train_dataset = dataset.lmdbDataset(root=trainroot, target_transform=one_hot)
# print(len(train_dataset))

test_dataset = dataset.lmdbDataset(
    root=valroot,
    transform=dataset.resizeNormalize((imgW, imgH)),
    target_transform=one_hot)

# 生成训练用数据
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batchSize,
    shuffle=True,
#     sampler=sampler,
    num_workers=2,
    collate_fn=dataset.alignCollate(
        imgH=imgH, imgW=imgW, keep_ratio=keep_ratio))

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=testSize, 
    num_workers=2,
    shuffle=False, )



nSamples:13254
nSamples:1325


In [19]:
# iter_data = iter(train_loader)
# %time x,y = next(iter_data)
# # print(y)
# x = x.numpy()
# X = x.reshape((-1, imgH, imgW, 1))
# Y = np.array(y)
# Length = int(imgW / 4) - 2
# batch = X.shape[0]
# X_train, Y_train = [
#     X, Y, np.ones(batch) * Length,
#     np.ones(batch) * n_len], np.ones(batch)
# X_train[0].shape,X_train[1].shape,X_train[2].shape,X_train[3].shape

In [20]:
# %timeit x,y = next(iter_data)


In [ ]:
j = 0

print('Start training!!')
for i in range(EPOCH_NUMS):
    for X, Y in train_loader:
        start = time.time()
        X = X.numpy()
        X = X.reshape((-1, imgH, imgW, 1))
#         print('Y ',Y)
        Y = np.array(Y)
        Length = int(imgW / 4) - 2
        batch = X.shape[0]
        X_train, Y_train = [
            X, Y, np.ones(batch) * Length,
            np.ones(batch) * n_len
        ], np.ones(batch)
#         print(Y_train)
        model.train_on_batch(X_train, Y_train)
        j +=1 
        if j % interval == 0:
            times = time.time() - start
            currentLoss_train = model.evaluate(X_train, Y_train)
            crrentLoss = 0
            for X,Y in test_loader:
                X = X.numpy()
                X = X.reshape((-1, imgH, imgW, 1))
                Y = Y.numpy()
                Y = np.array(Y)
                batch = X.shape[0]
                X_val, Y_val = [
                    X, Y, np.ones(batch) * Length,
                    np.ones(batch) * n_len
                ], np.ones(batch)
                crrentLoss += model.test_on_batch(X_val, Y_val)
            print('Learning rate is: ', LEARNING_RATE)
            now_time = time.strftime('%Y/%m/%d-%H:%M:%S',
                                     time.localtime(time.time()))
            print('Time: [%s]--Step/Epoch/Total: [%d/%d/%d]' % (now_time, j, i,
                                                                EPOCH_NUMS))
            print('\tTraining Loss is: [{}]'.format(currentLoss_train))
            print('\tVal Loss is: [{}]'.format(crrentLoss))
            print('\tSpeed is: [{}] Samples/Secs'.format(interval / times))
            path = MODEL_PATH + '/num_char_%06d_10.h5'%(crrentLoss*10)
            with open(LOG_FILE, mode='a') as log_file:
                log_str = now_time + '----global_step:' + str(
                    j) + '----loss:' + str(loss) + '\n'
                log_file.writelines(log_str)
            log_file.close()
            print('\tWriting to the file: log.txt')
            if crrentLoss < loss:
                loss = crrentLoss
                print("\tSave model to disk: {}".format(path))
                model.save(path)  


Start training!!
32/32 [==============================] - 1s 38ms/step
Learning rate is:  0.001
Time: [2018/10/23-06:52:10]--Step/Epoch/Total: [200/0/100000]
	Training Loss is: [0.02093873918056488]
	Val Loss is: [8.46971432864666]
	Speed is: [524.312577152054] Samples/Secs
	Writing to the file: log.txt
	Save model to disk: /mnt/wuwenhui/git_ocr_project/keras_crnn/save_model/num_char_000084_10.h5
32/32 [==============================] - 0s 3ms/step
Learning rate is:  0.001
Time: [2018/10/23-06:53:36]--Step/Epoch/Total: [400/0/100000]
	Training Loss is: [0.04130443185567856]
	Val Loss is: [7.997243277728558]
	Speed is: [517.2185916371791] Samples/Secs
	Writing to the file: log.txt
	Save model to disk: /mnt/wuwenhui/git_ocr_project/keras_crnn/save_model/num_char_000079_10.h5
32/32 [==============================] - 0s 3ms/step
Learning rate is:  0.001
Time: [2018/10/23-06:55:00]--Step/Epoch/Total: [600/1/100000]
	Training Loss is: [0.014036129228770733]
	Val Loss is: [8.19473285973072]
	

Learning rate is:  0.001
Time: [2018/10/23-07:29:36]--Step/Epoch/Total: [5600/13/100000]
	Training Loss is: [0.008648285642266273]
	Val Loss is: [8.123429343104362]
	Speed is: [511.0181365394128] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 3ms/step
Learning rate is:  0.001
Time: [2018/10/23-07:30:59]--Step/Epoch/Total: [5800/13/100000]
	Training Loss is: [0.004438498057425022]
	Val Loss is: [8.148283991962671]
	Speed is: [430.33818189653135] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 3ms/step
Learning rate is:  0.001
Time: [2018/10/23-07:32:22]--Step/Epoch/Total: [6000/14/100000]
	Training Loss is: [0.0025873971171677113]
	Val Loss is: [8.13744181022048]
	Speed is: [401.6682292349991] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 3ms/step
Learning rate is:  0.001
Time: [2018/10/23-07:33:45]--Step/Epoch/Total: [6200/14/100000]
	Training Loss is: [0.001080782851

In [ ]:
x,y = iter(train_loader)